In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from sklearn.preprocessing import MultiLabelBinarizer
from categories import *
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os
from pathlib import Path

In [ ]:
table_path = os.path.join(Path(__file__).parent, 'video_data', 'imageTable.csv')
imagedir = os.path.join(Path(__file__).parent, 'videos', 'images')

Create the Scikit-learn MultiLabelBinarizer

In [ ]:
labels = [("menu",),tuple(decode_styles.values()),tuple(decode_themes.values()),("day","night")]
classes = [x for t in labels for x in t ]
mlb = MultiLabelBinarizer(classes=classes)
mlb.fit(labels)


Read the dataset

In [ ]:
df = pd.read_csv(table_path)

This function wraps flow_from_dataframe to accept a multilabelbinaraizer

In [ ]:
def multilabel_flow_from_dataframe(data_generator, mlb):
    print("mffd called")
    for x, y in data_generator:
        assert isinstance(mlb, MultiLabelBinarizer), \
               "MultiLabelBinarizer is required."
        indices = y.astype(np.int).tolist()
        rows = df.iloc[indices]
        tup = rows.apply(lambda r: strip_tuple((r['Style'],r['Theme'],r['Time'])), axis=1)
        y_multi = mlb.transform(
             tup
        )
        yield x, y_multi

In [ ]:
def get_generator(frame,mlb):
    gen = image_data_generator.flow_from_dataframe(
        dataframe=frame,
        directory=imagedir,
        x_col='Filename',
        y_col='index',
        class_mode='raw',
        target_size=(512,512),
        batch_size=32
    )
    return multilabel_flow_from_dataframe(gen, mlb)

The CNN model

In [ ]:
def prepare_model():
    model = keras.Sequential([
        # Block One
        layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
                    input_shape=[512,512, 3]),
        layers.MaxPool2D(),

        # Block Two
        layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPool2D(),

        # Block Three
        layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
        layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPool2D(),

        # Head
        layers.Flatten(),
        layers.Dense(6, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(18, activation='sigmoid'),
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(epsilon=0.01),
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model

Setup checkpoints and early stop

In [ ]:
checkpointer = ModelCheckpoint(
        filepath=os.path.join('model_data', 'checkpoints', 'theme_classifier' + \
            '.{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=True)

early_stopper = EarlyStopping(patience=5)

tb = TensorBoard(log_dir=os.path.join('model_data', 'logs'))

timestamp = time.time()
csv_logger = CSVLogger(os.path.join('data', 'logs', 'theme_classifier-training-' + \
    str(timestamp) + '.log'))

Set up the variables

In [ ]:
train_df = df.loc[df['Train'].values]
valid_df = df[(df['Train'].values) == False]

train_generator = get_generator(train_df,mlb)
valid_generator = get_generator(valid_df,mlb)

Now run the model

In [ ]:
model = prepare_model()

model.fit(
    train_generator,
    validation_data = valid_generator,
    callbacks=[tb, early_stopper, csv_logger, checkpointer],
    epochs = 5
)